The Following code is inspired from [course.fast.ai](course.fast.ai). 
This Model takes in the following inputs:
1. No of Elements to create a Model
2. Name of the Elements to create the Model

Please note that the code downloads the images of the model so internet connectivity is a must, also please install all necessary libraries and packages before running the code. The code is working perfectly as of 30/04/2024

In [ ]:
from duckduckgo_search import DDGS
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *

#If you have pytorch installed you can uncomment the following code:
# import torch
# if torch.cuda.is_available():
#     device = torch.device("cuda")
# else:
#     device = torch.device("cpu")

ddgs = DDGS()

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddgs.images(keywords=term, max_results=max_images)).itemgot('image')


In [ ]:
no_of_models = int(input("Enter the number of elements you want to train the model with: "))
searches = []
for i in range(no_of_models):
    temp = input(f"Enter the name of your Element {i+1}: ")
    searches.append(temp)

In [ ]:
path = Path('training_model')
from time import sleep
import shutil

def clean_directory(dir_path):
    for file_or_dir in os.listdir(dir_path):
        file_or_dir_path = os.path.join(dir_path,file_or_dir)
        if os.path.isfile(file_or_dir_path):
            os.remove(file_or_dir_path)
        elif os.path.isdir(file_or_dir_path):
            shutil.rmtree(file_or_dir_path)

if path.exists():
    clean_directory(path)

for o in searches:
    '''This block of code basically downloads images of the elements specified on the device, and sorts them into different folders in the
    "training_model" folder, if the folder doesnt exist, it creates it, if it does it overwrites the content in it. '''
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(2)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(2)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(2)
    resize_images(path/o, max_size=400, dest=path/o)

#Removes the failed images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

# len(failed) Uncomment this line to check the number of images failed


In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=10)

#Displays few images downloaded with the element name specified so you can verify it

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)
print("Your Model is Ready for testing")

In [ ]:
#Input the path of the testing image below
test_image_name = "cheetah_training_1.jpg"
is_what,_,probs = learn.predict(PILImage.create(test_image_name))
print(f"This is a: {is_what}.")